<a href="https://colab.research.google.com/github/kiddhcx/ECE720/blob/main/MODE_ON_PCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/My Drive/MODE')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Sun Mar 27 02:00:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    32W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip uninstall tensorflow-gpu
!pip install tensorflow==2.2
!pip uninstall keras
!pip install keras==2.3.1
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install keras-efficientnets

In [13]:
from keras.utils import HDF5Matrix
import h5py
import numpy as np
import tensorflow as tf
import keras as k
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Lambda, Dense, Flatten, Dropout, Reshape, Conv2D, Activation, GlobalAveragePooling2D
from keras.models import Model
from keras.models import Sequential
from keras_efficientnets import EfficientNetB0
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import matplotlib.pyplot as plt

# personal modules
from distance_metrics import bhattacharyya
from eval import get_eval_datasets, replicate_model
from layer_selection import (Forward_Layer_Select, 
                                calculate_acc_for_labels, 
                                get_overfit_labels, 
                                get_underfit_labels,
                                get_faultiest_label)
from heatmap import get_heatmaps
from input_selection import select_next_inputs
from visualizations import (get_plot, 
                               visualize_array, 
                               get_samples_for_label,
                               sample_misclassifications, 
                               viz_heatmaps_for_correct_prediction, 
                               viz_heatmaps_for_false_positives, 
                               viz_heatmaps_for_false_negatives,
                               DHCI_i_k,
                               DHMI_i,
                               DHWI_i,
                               viz_most_least_similar,
                               viz_most_similar,
                               viz_least_similar)

# plot defaults
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'




In [6]:
x_train = HDF5Matrix('/content/drive/My Drive/MODE/pcamv1/camelyonpatch_level_2_split_train_x.h5', 'x')
f_train = h5py.File('/content/drive/My Drive/MODE/pcamv1/camelyonpatch_level_2_split_train_y.h5', 'r')
dset = f_train['y']
y_train = np.array(dset[:,:,:,:])
y_train = y_train.reshape(262144,1)

x_test = HDF5Matrix(r'/content/drive/My Drive/MODE/pcamv1/camelyonpatch_level_2_split_test_x.h5', 'x')
f_test = h5py.File(r'/content/drive/My Drive/MODE/pcamv1/camelyonpatch_level_2_split_test_y.h5', 'r')
y_test_set = f_test['y']
y_test = np.array(y_test_set[:,:,:,:])
y_test = y_test.reshape(32768,1)

x_bug_fixes= HDF5Matrix(r'/content/drive/My Drive/MODE/pcamv1/camelyonpatch_level_2_split_valid_x.h5', 'x')
f_bug_fixes = h5py.File(r'/content/drive/My Drive/MODE/pcamv1/camelyonpatch_level_2_split_valid_y.h5', 'r')
y_bug_fixes_set = f_bug_fixes['y']
y_bug_fixes= np.array(y_bug_fixes_set[:,:,:,:])
y_bug_fixes = y_bug_fixes.reshape(32768,1)

/usr/local/lib/python3.7/dist-packages/keras/utils/io_utils.py:60: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(datapath)


In [7]:
theta = 0.90
gamma = 0.10
alpha = 0.25
similarity_threshold = 0.01


In [8]:
x_train = x_train[0:62144]
y_train = y_train[0:62144]
print('Training data shape: ', x_train.shape)
print('Training labels shape: ', y_train.shape)

print('Bug Fixing data shape: ', x_bug_fixes.shape)
print('Bug Fixing labels shape: ', y_bug_fixes.shape)

print('Test data shape: ', x_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (62144, 96, 96, 3)
Training labels shape:  (62144, 1)
Bug Fixing data shape:  (32768, 96, 96, 3)
Bug Fixing labels shape:  (32768, 1)
Test data shape:  (32768, 96, 96, 3)
Test labels shape:  (32768, 1)


In [14]:
DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}

def swish(x):
    return x * tf.sigmoid(x)

model = EfficientNetB0(input_shape = (96,96,3), weights = 'imagenet', include_top = False)
x = model.output
x = GlobalAveragePooling2D(name = 'avg_pool')(x)
x = Dropout(0.2)(x)
x = Dense(1, activation = swish, kernel_initializer = DENSE_KERNEL_INITIALIZER)(x)
prediction = Activation('sigmoid')(x)
model = Model(inputs=model.input, outputs=prediction)

In [15]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
datagen = ImageDataGenerator(
    preprocessing_function=lambda x: x/255.)
valid_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: x/255. )

In [17]:
model.fit_generator(
  datagen.flow(x_train, y_train, batch_size=128),
  epochs=5
  )

Epoch 1/5
486/486 [==============================] - 140s 287ms/step - loss: 0.6104 - accuracy: 0.7143
Epoch 2/5
486/486 [==============================] - 124s 254ms/step - loss: 0.4745 - accuracy: 0.8658
Epoch 3/5
486/486 [==============================] - 123s 253ms/step - loss: 0.4385 - accuracy: 0.9035
Epoch 4/5
486/486 [==============================] - 123s 253ms/step - loss: 0.4359 - accuracy: 0.9035
Epoch 5/5
486/486 [==============================] - 123s 254ms/step - loss: 0.4180 - accuracy: 0.9090


In [18]:
# predict train (to get distributions)
y_pred_train = model.predict(x_train)

# predict test (to get distributions)
y_pred = model.predict(x_test)

In [21]:
(loss, accuracy) = model.evaluate_generator(
   datagen.flow(x_test, y_test, batch_size=128), 
   verbose=1)

print('loss={:.4f}, accuracy: {:.4f}%'.format(loss, accuracy * 100))

256/256 [==============================] - 14s 55ms/step
loss=0.5106, accuracy: 80.0415%
